In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import differential_evolution

In [2]:
#read data
d = pd.read_csv('C:/Users/cape159/Documents/pracovni/data_statistika/kopackuv_grant/mixing_experiment/DB_concept/Hasan_Jolanta/hasan_jolanta.csv')

print(d)

   Soil Status   time   G12  G12init  G13  G13init  CO212  CO213      DOC12  \
0    PL      A    0.1  72.3     97.7  2.0      2.7    0.7    0.0  22.100000   
1    PL      A   12.0  14.0     97.7  0.4      2.7   22.1    0.5        NaN   
2    PL      A   24.0   0.3     97.7  0.0      2.7   36.1    0.9  21.204962   
3    PL      A   48.0   0.2     97.7  0.0      2.7   50.3    1.1  19.468059   
4    PL      A   72.0   0.0     97.7  0.0      2.7   61.3    1.3  19.662000   
5    CT      A    0.1  64.0     89.0  1.8      2.5    0.5    0.0  18.577000   
6    CT      A   12.0  29.7     89.0  0.8      2.5   10.3    0.2        NaN   
7    CT      A   24.0   5.3     89.0  0.1      2.5   20.8    0.5  21.628389   
8    CT      A   48.0   0.2     89.0  0.0      2.5   37.9    0.9  18.054769   
9    CT      A   72.0   0.0     89.0  0.0      2.5   44.5    1.1  17.284000   
10   PL      N    0.1  78.0    105.3  2.1      2.9    0.2    0.0        NaN   
11   PL      N   12.0  74.4    105.3  2.0      2.9  

In [3]:
# define Monod model
def Monodmodel (y, t, pars):
    #define initial states
    MBC12=y[0];    G12=y[1];   DOC12=y[2];   CO212=y[3]
    MBC13=y[4];    G13=y[5];   DOC13=y[6];   CO213=y[7]
        
    #define parameters
    Vmaxg=pars[0];     Kmg=pars[1];   Vmax=pars[2];      Km=pars[3]
    CUEg=pars[4];      CUEs=pars[5];  kb=pars[6];        kec=pars[7]
        
    
    #G uptake rate total
    Cug_tot=Vmaxg*(MBC12+MBC13)*(G12+G13)/(G12+G13+Kmg*(1+(DOC12+DOC13)/Km))
        
    #DOC uptake rate total
    Cu_tot=Vmax*(MBC12+MBC13)*(DOC12+DOC13)/(DOC12+DOC13+Km*(1+(G12+G13)/Kmg))
      
    #define 13C atm% scaling factors
    Gatm=G13/(G12+G13)
    DOCatm=DOC13/(DOC12+DOC13)
    
    #derivatives
    dMBC12dt=CUEg*Cug_tot*(1-Gatm)+CUEs*Cu_tot*(1-DOCatm)-kb*MBC12
    dG12dt=-Cug_tot*(1-Gatm)
    dDOC12dt=-Cu_tot*(1-DOCatm)+kec*kb*MBC12
    dCO212dt=(1-CUEg)*Cug_tot*(1-Gatm) + (1-CUEs)*Cu_tot*(1-DOCatm)
    dMBC13dt=CUEg*Cug_tot*Gatm+CUEs*Cu_tot*DOCatm-kb*MBC13
    dG13dt=-Cug_tot*Gatm
    dDOC13dt=-Cu_tot*DOCatm+kec*kb*MBC13
    dCO213dt=(1-CUEg)*Cug_tot*Gatm + (1-CUEs)*Cu_tot*DOCatm
    
            
    return dMBC12dt, dG12dt, dDOC12dt, dCO212dt, dMBC13dt, dG13dt, dDOC13dt, dCO213dt;

In [4]:
#define a function returning ode results with additional calculations
def calc (model, pars, t, y0):
    #first solve the model
    y = odeint(model,y0,t, args=(pars,))
    
    #calculate Cmic12 and add it to y frame
    Cmic12 = pars[7] * y[:, 0] 
    #calculate Cmic13 and add it to y frame
    Cmic13 = pars[7] * y[:, 4] 
    
    yhat = np.concatenate((y[:, 1].reshape(5,1),#G12
                           y[:, 2].reshape(5,1),#DOC12
                           y[:, 3].reshape(5,1),#CO212
                           y[:, 5].reshape(5,1),#G13
                           y[:, 6].reshape(5,1),#DOC13
                           y[:, 7].reshape(5,1),#CO213
                           Cmic12.reshape(5,1),
                           Cmic13.reshape(5,1)), axis=1)
    
    return yhat

In [5]:
#create the minimization function
def obj_fun (x):
    #define parameters
    pars = x
    
    #initial conditions
    MBC12init = data.Cmic12init[0]/pars[7]
    MBC13init = data.Cmic13init[0]/pars[7]
    G12init = data.G12init[0]
    G13init = data.G13init[0]
    DOC12init = data.DOC12init[0]
    DOC13init = data.DOC13init[0]
        
    y0 = np.array([MBC12init, G12init, DOC12init,0,
                   MBC13init, G13init, DOC13init,0])
    
    #times
    t = data.time
    
    #use the function to get DOC, respiration rate and Cmic
    yhat_full = calc(Monodmodel, pars, t, y0)
         
    #observations
    obs=np.concatenate((np.array([data.G12]).reshape(5,1),
                        np.array([data.DOC12]).reshape(5,1),
                        np.array([data.CO212]).reshape(5,1),
                        np.array([data.G13]).reshape(5,1),
                        np.array([data.DOC13]).reshape(5,1),
                        np.array([data.CO213]).reshape(5,1),
                        np.array([data.Cmic12]).reshape(5,1),
                        np.array([data.Cmic13]).reshape(5,1)), 
                     axis=1)
    
    #weights
    weights=np.concatenate((np.nanmean(data.G12).repeat(5).reshape(5,1),
                            np.nanmean(data.DOC12).repeat(5).reshape(5,1),
                            np.nanmean(data.CO212).repeat(5).reshape(5,1),
                            np.nanmean(data.G13).repeat(5).reshape(5,1),
                            np.nanmean(data.DOC13).repeat(5).reshape(5,1),
                            np.nanmean(data.CO213).repeat(5).reshape(5,1),
                            np.nanmean(data.Cmic12/5).repeat(5).reshape(5,1),
                            np.nanmean(data.Cmic13).repeat(5).reshape(5,1)), 
                       axis=1)
                
          
    out=np.nansum(((yhat_full-obs)/weights)**2)
          
    return out

In [6]:
#create goodness of fit function
def goodness (x):
    #define parameters
    pars = x
    
    #initial conditions
    MBC12init = data.Cmic12init[0]/pars[7]
    MBC13init = data.Cmic13init[0]/pars[7]
    G12init = data.G12init[0]
    G13init = data.G13init[0]
    DOC12init = data.DOC12init[0]
    DOC13init = data.DOC13init[0]
        
    y0 = np.array([MBC12init, G12init, DOC12init,0,
                   MBC13init, G13init, DOC13init,0])
    
    #times
    t = data.time
    
    #use the function to get DOC, respiration rate and Cmic
    yhat_full = calc(Monodmodel, pars, t, y0)
         
    #observations
    obs=np.concatenate((np.array([data.G12]).reshape(5,1),
                        np.array([data.DOC12]).reshape(5,1),
                        np.array([data.CO212]).reshape(5,1),
                        np.array([data.G13]).reshape(5,1),
                        np.array([data.DOC13]).reshape(5,1),
                        np.array([data.CO213]).reshape(5,1),
                        np.array([data.Cmic12]).reshape(5,1),
                        np.array([data.Cmic13]).reshape(5,1)), 
                     axis=1)                   
          
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = 2*8 - 2*ll
    
    out = np.array([R2, ll, AIC])
          
    return out

In [7]:
#Plesne aerobni
data = d[(d.Soil=='PL') & (d.Status=='A')]
data = data.reset_index(drop=True)

optimum_PA=differential_evolution(obj_fun, [(0.0001, 10), (0.1, 500), 
                                            (0.0001, 10), (0.1, 500),
                                            (0.05,0.9), (0.05,0.9), (0.0001, 1), (0,1)], 
                                  polish=True, maxiter=1000000)

print(optimum_PA)

C:\Users\cape159\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


     fun: 5.011688442652362
     jac: array([ 3.73390208e-04,  3.59712260e-05,  2.39808173e-05, -6.21724894e-07,
        8.36664071e-05,  3.30752670e-01,  8.75033379e-03, -2.88657986e-04])
 message: 'Optimization terminated successfully.'
    nfev: 8217
     nit: 65
 success: True
       x: array([2.49787791e-01, 3.04506626e+02, 2.41564530e-02, 4.66275127e+02,
       6.50735003e-01, 5.00000000e-02, 2.15227385e-03, 2.84543810e-01])


In [8]:
print(goodness(optimum_PA.x))
np.savetxt('PL_monod.csv', optimum_PA.x.reshape(1,8), delimiter=",")

[ 0.95460564 -0.81709849 17.63419698]


In [9]:
#Certovo aerobni
data = d[(d.Soil=='CT') & (d.Status=='A')]
data = data.reset_index(drop=True)

optimum_CA=differential_evolution(obj_fun, [(0.0001, 10), (0.1, 500), 
                                            (0.0001, 10), (0.1, 500),
                                            (0.05,0.9), (0.05,0.9), (0.0001, 1), (0,1)], 
                                  polish=True, maxiter=1000000)
print(optimum_CA)

     fun: 5.358350048721805
     jac: array([ 5.32907052e-06,  8.95283847e-05, -9.40580946e-05, -2.13162821e-06,
        1.35003120e-05,  3.20312488e-01,  2.69002598e-03, -4.06785716e-05])
 message: 'Optimization terminated successfully.'
    nfev: 14214
     nit: 114
 success: True
       x: array([1.89422380e-01, 2.85710137e+02, 1.60345686e-02, 2.59207492e+02,
       7.13778156e-01, 5.00000000e-02, 1.67540922e-03, 4.00701715e-01])


In [10]:
print(goodness(optimum_CA.x))
np.savetxt('CT_monod.csv', optimum_CA.x.reshape(1,8), delimiter=",")

[ 0.95310174 -0.84416867 17.68833735]
